In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import math
import pandas as pd
from pydantic import BaseModel, Field
from typing import List, Optional
import yaml
from scipy.io import arff

from sklearn.decomposition import KernelPCA
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# synthetic datasets

In [2]:
def make_alternate_stripes():

    data = []

    for i in [np.pi * i for i in [-3, -1, 1, 3, 5]]:
        noise = np.random.randn(200, 1)*0.1
        x = np.expand_dims(np.repeat(i, 200), axis=1) + noise
        y = noise

        concat = np.concatenate((x, y), axis=1)
        data.append(concat)

    for i in [np.pi * i for i in [-4, -2, 0, 2, 4]]:
        noise = np.random.randn(200, 1)*0.1
        x = np.expand_dims(np.repeat(i, 200), axis=1) + noise
        y = noise

        concat = np.concatenate((x, y), axis=1)
        data.append(concat)

    data_x = np.reshape(np.array(data), (2000, 2))

    return data_x


def fibonacci_sphere(r, samples=1000, mu=0, sigma=0.05):
    points = []
    phi = math.pi * (3. - math.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from k=1 to -k=1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius

        points.append((x, y, z))

        noise = np.random.randn(1000, 3)
        noise = noise * sigma + mu

    return np.array(points) * r + noise


def make_two_spheres():
    sphere_one = fibonacci_sphere(0.1)
    sphere_two = fibonacci_sphere(0.5)

    x = np.concatenate((sphere_one, sphere_two), axis=0)

    return x


def circles_data(x0=0, y0=0):
    np.random.seed(0)
    x, y = datasets.make_circles(n_samples=2000, factor=0.1, noise=0.05)
    x[:, 0] += x0
    x[:, 1] += y0

    return x

# load config

In [3]:
class ConfigDataset(BaseModel):
    num_features: int
    synthetic_bool: bool
    categorical_features: Optional[List[int]] = Field(default_factory=list)
    numerical_features: Optional[List[int]] = Field(default_factory=list)

In [4]:
def dataset_config_load(dataset: str) -> ConfigDataset:
    """load config file into pydantic object"""
    file_path = "../datasets_config.yaml"
    
    with open(file_path) as file:
        config_data = yaml.safe_load(file)
        config_data = config_data[dataset]

    return ConfigDataset(**config_data)

# load data

In [5]:
def read_arff(path):
    data, meta = arff.loadarff(path)
    data = pd.DataFrame(data)
    return data


def preprocess_data(data: pd.DataFrame, dataset: str) -> tuple[pd.DataFrame, np.array]:
    type_class = np.zeros(shape=(1, data.shape[0]))
    if dataset == "abalone":
        type_class = data["sex"]

    elif dataset in ["phoneme", "breast_cancer"]:
        type_class = data["Class"]

    elif dataset in ["wine", "ionosphere", "german_credit", "dermatology", "heart-statlog"]:
        type_class = data["class"]

    return data, type_class


def load_data(dataset: str) -> pd.DataFrame:
    if dataset == "spheres":
        data = make_two_spheres()

    elif dataset == "circles":
        data = circles_data()

    elif dataset == "alternate_stripes":
        data = make_alternate_stripes()

    else:
        path = f"../datasets/{dataset}.arff"
        data = read_arff(path)

    return pd.DataFrame(data)

In [22]:
def get_split_indices(data: np.array, random_seed: int, val_prop: float = 0.2) -> tuple[np.array, np.array]:
    np.random.seed(random_seed)
    n = data.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    train_indices = indices[int(n * val_prop):]
    val_indices = indices[:int(n * val_prop)]

    return train_indices, val_indices

In [23]:
def transform_data_onehot(data: pd.DataFrame, object_indices: list[int]) -> tuple[pd.DataFrame, list]:
    object_indices = np.array(object_indices)
    data_to_one_hot = data.iloc[:, object_indices]

    num_cols_per_categories = list(data_to_one_hot.nunique())
    cols_to_remove = data_to_one_hot.columns

    data_to_one_hot = pd.get_dummies(data_to_one_hot, columns=cols_to_remove, dtype=int)

    data = data.drop(columns=cols_to_remove, inplace=False)
    num_cols_per_categories = num_cols_per_categories + [1] * data.shape[1]
    data = pd.concat((data_to_one_hot, data), axis=1)

    return data, num_cols_per_categories

In [24]:
# def transform_data_onehot(data: pd.DataFrame, object_indices: list[int]) -> tuple[pd.DataFrame, list[int]]:
#     object_indices = np.array(object_indices)
#     columns = data.columns[object_indices]

#     for col in columns:
#         data[col], _ = pd.factorize(data[col])

#     num_cols_per_categories = [k=1] * data.shape[k=1]

#     return data, num_cols_per_categories

In [25]:
def load_data_and_preprocess(dataset: str):

    dataset_config = dataset_config_load(dataset)
    x = load_data(dataset)
    x = x.dropna()

    classes = np.zeros(x.shape[0])

    if dataset not in ["circles", "spheres", "alternate_stripes"]:
        x, classes = preprocess_data(x, dataset)
        classes, mapping = pd.factorize(classes)

    # transform categorical columns to one-hot encoded.
    if dataset_config.categorical_features:
        x, num_features_per_network = transform_data_onehot(x,
                                                            dataset_config.categorical_features
                                                            )
    else:
        num_features_per_network = np.array([1] * x.shape[1])

    train_indices, val_indices = get_split_indices(x, run_index)
    train_x = x.iloc[train_indices].values
    val_x = x.iloc[val_indices].values

    return train_x, val_x

# Kernel PCA and plot the principal components

In [31]:
plot = False
num_components = 1
kernel_list = ["rbf", "cosine", "poly", "sigmoid", "linear"] # linear is regular PCA

dataset_list = ["alternate_stripes",
                "circles",
                "spheres", 
                "wine", 
                "heart-statlog",
                "ionosphere", 
                "breast_cancer", 
                "german_credit"]

In [32]:
data_dict = {}

for dataset in dataset_list:
    print(f"Computing for dataset {dataset}...")
    data_dict[dataset] = {}
    
    for ke in kernel_list:
        print(f"Computing for kernel {ke}...")
        data_dict[dataset][ke] = []
        across_runs_list = []
        for run_index in range(15):
    
            train_x, val_x = load_data_and_preprocess(dataset)
            scaler = StandardScaler()
            scaler.fit(train_x)
            train_x = scaler.transform(train_x)
            val_x = scaler.transform(val_x)
            
            kernel_pca = KernelPCA(n_components=train_x.shape[1], kernel=ke, gamma=1, alpha=0.1, degree=2)
            kernel_pca.fit(train_x)
            X_kpca = kernel_pca.transform(train_x)
            X_kpca_val = kernel_pca.transform(val_x)
    
            if plot:
                # Plot results
                fig, ax = plt.subplots()
                ax.grid(True)
            
                ax.scatter(X_kpca[:, 0], X_kpca[:, 1], c="blue", s=20, edgecolor="k")
                low_y_lim, high_y_lim = plt.gca().get_ylim()
                low_x_lim, high_x_lim = plt.gca().get_xlim()
                ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
                ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
                ax.tick_params(axis='both', which='major', labelsize=14)
                ax.set_xlabel(r"$\widetilde{z}_1$", size=15)
                ax.set_ylabel(r"$\widetilde{z}_2$", size=15)
                plt.savefig(f'original_{dataset}_{ke}.pdf', dpi=300, bbox_inches='tight')
                plt.show()
    
            training_explained_var = kernel_pca.eigenvalues_/np.sum(kernel_pca.eigenvalues_)
            # For training data, compute variance along each component. Check that this is the same as the above => it is.
            # train_var = np.var(X_kpca, axis=0)
            # train_exp_var_ratio = train_var / np.sum(train_var)
            # print("Validation projected explained variance ratio:", train_exp_var_ratio)
    
            # For validation data, compute variance along each component
            val_var = np.var(X_kpca_val, axis=0)
            val_exp_var_ratio = val_var / np.sum(val_var)
            # print("Validation projected explained variance ratio:", val_exp_var_ratio)

            across_runs_list.append((training_explained_var[:num_components], val_exp_var_ratio[:num_components]))
            
        train_statistics = np.mean([val[0] for val in across_runs_list]), np.std([val[0] for val in across_runs_list])
        val_statistics = np.mean([val[1] for val in across_runs_list]), np.std([val[1] for val in across_runs_list])
    
        data_dict[dataset][ke].append((train_statistics, val_statistics))


Computing for dataset alternate_stripes...
Computing for kernel rbf...


/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Libr

Computing for kernel cosine...


/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Libr

Computing for kernel poly...


...

/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Libr

Computing for kernel sigmoid...


/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Libr

Computing for kernel linear...


/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/thomas/Library/Caches/pypoetry/virtualenvs/es-pca-6NHe3mCZ-py3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/thomas/Libr

In [33]:
print(data_dict)

{'alternate_stripes': {'rbf': [((0.5729248903665268, 0.006134530354470467), (0.571392426020802, 0.020843365550841004))], 'cosine': [((0.550560874558379, 0.004838640878968761), (0.5465976754796137, 0.022165978972471686))], 'poly': [((0.5195479206983351, 0.01233040612070574), (0.5051300068587462, 0.043591386405435584))], 'sigmoid': [((0.5382406883871202, 0.0029550794578066847), (0.5346436601533479, 0.024133661153199235))], 'linear': [((0.5142040381904613, 0.006733305602740048), (0.5183384216918062, 0.029823584399537178))]}, 'circles': {'rbf': [((0.7850471038290692, 0.0032658363484075804), (0.8030667515119124, 0.00992884840413113))], 'cosine': [((0.506171976481016, 0.0034205831554296436), (0.4903379414843132, 0.014295996243029735))], 'poly': [((0.5045217835222313, 0.0027584123286982713), (0.49344599893441343, 0.03369525631317454))], 'sigmoid': [((0.5055212945276868, 0.0019444120367324012), (0.46666713668160587, 0.017042667891099587))], 'linear': [((0.5042792405430389, 0.002213556994760224